<a href="https://colab.research.google.com/github/jimitogni/ml_study/blob/master/monkeys_bread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
os.environ['KAGLE_USERNAME']= "jimitogni"
os.environ['KAGLE_KEY']= "6ba3f466a063e70addd26514bb2cf41d"

In [1]:
%%shell

export KAGGLE_USERNAME=jimitogni
export KAGGLE_KEY=6ba3f466a063e70addd26514bb2cf41d

kaggle datasets download slothkong/10-monkey-species

 95% 521M/547M [00:10<00:00, 30.2MB/s]
100% 547M/547M [00:10<00:00, 53.0MB/s]


In [2]:
!unzip 10-monkey-species.zip

Archive:  10-monkey-species.zip
  inflating: monkey_labels.txt       
  inflating: training/training/n0/n0018.jpg  
  inflating: training/training/n0/n0019.jpg  
  inflating: training/training/n0/n0020.jpg  
  inflating: training/training/n0/n0021.jpg  
  inflating: training/training/n0/n0022.jpg  
  inflating: training/training/n0/n0023.jpg  
  inflating: training/training/n0/n0024.jpg  
  inflating: training/training/n0/n0025.jpg  
  inflating: training/training/n0/n0026.jpg  
  inflating: training/training/n0/n0027.jpg  
  inflating: training/training/n0/n0028.jpg  
  inflating: training/training/n0/n0029.jpg  
  inflating: training/training/n0/n0030.jpg  
  inflating: training/training/n0/n0031.jpg  
  inflating: training/training/n0/n0032.jpg  
  inflating: training/training/n0/n0033.jpg  
  inflating: training/training/n0/n0034.jpg  
  inflating: training/training/n0/n0035.jpg  
  inflating: training/training/n0/n0036.jpg  
  inflating: training/training/n0/n0037.jpg  
  inflatin

In [3]:
from keras.applications import MobileNet
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [4]:
img_rows, img_cols = 224, 224

MobileNet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #largura, altura, 3=RGB colorido

for layer in MobileNet.layers:
  layer.trainable=False

# lista as camadas que definimos para usar com entrada 0 InputLayer False
# todos estão como false, pois não vamos treina-los, quando utilizamos "layer.trainable=False" acima
for (i, layer) in enumerate(MobileNet.layers): #pega os layers da mobilenet e coloca em payer e o i fica como index
  print(str(i),layer.__class__.__name__,layer.trainable)

17227776/17225924 [==============================] - 1s 0us/step
0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False


In [0]:
#criando o nosso top layer
def addTopModelMobileNet(bottom_model, num_classes):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(1024, activation='relu')(top_model)
  top_model = Dense(1024, activation='relu')(top_model)
  top_model = Dense(512, activation='relu')(top_model)
  top_model = Dense(num_classes, activation='softmax')(top_model)

  return top_model

In [6]:
num_classes = 10

FC = addTopModelMobileNet(MobileNet, num_classes) #MobileNet = bottom_model
model = Model(input=MobileNet.input, output=FC) #bottom layers, top layer

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  after removing the cwd from sys.path.


In [0]:
train_data_dir = '/content/training/training/'
validation_data_dir = '/content/validation/validation/'

In [8]:
#data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

#nao faz data aug
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(img_rows, img_cols),
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [9]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('macacos_modelo.h5',
                             monitor='val_loss',
                             mode=min,
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,
                          restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.0001)

callbacks = [checkpoint, earlystop, learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:680: RuntimeWarning: ModelCheckpoint mode <built-in function min> is unknown, fallback to auto mode.
  RuntimeWarning)


In [0]:
nb_train_samples = 1098
nb_validation_samples = 272

epochs = 25
batch_size = 32

In [11]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=nb_train_samples//batch_size,
                              epochs=epochs,
                              callbacks=callbacks,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
34/34 [==============================] - 40s 1s/step - loss: 1.4354 - accuracy: 0.5319 - val_loss: 0.3803 - val_accuracy: 0.8555

Epoch 00001: val_loss improved from inf to 0.38028, saving model to macacos_modelo.h5
Epoch 2/25
34/34 [==============================] - 37s 1s/step - loss: 0.4784 - accuracy: 0.8302 - val_loss: 0.3483 - val_accuracy: 0.8750

Epoch 00002: val_loss improved from 0.38028 to 0.34826, saving model to macacos_modelo.h5
Epoch 3/25
34/34 [==============================] - 38s 1s/step - loss: 0.3329 - accuracy: 0.8865 - val_loss: 0.4332 - val_accuracy: 0.8750

Epoch 00003: val_loss did not improve from 0.34826
Epoch 4/25
34/34 [==============================] - 36s 1s/step - loss: 0.2941 - accuracy: 0.9053 - val_loss: 0.2117 - val_accuracy: 0.9583

Epoch 00004: val_loss improved from 0.34826 to 0.21168, saving model to macacos_modelo.h5
Epoch 5/25
34/34 [==============================] - 37s 1s/step - loss: 0.2429 - accuracy: 0.9146 - val_loss: 0.2387 - 

In [0]:
# TESTANDO

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

model = load_model('/content/macacos_modelo.h5')

img_rows, img_cols = 224, 224

In [0]:
class_labels = [
	'mantled_howler', 
	'patas_monkey', 
	'bald_uakari', 
	'japanese_macaque', 
	'pygmy_marmoset', 
	'white_headed_capuchin',
	'silvery_marmoset',
	'common_squirrel_monkey', 
	'black_headed_night_monkey',
	'nilgiri_langur' 
	]

In [0]:
def check(patch):
  #prediction
  img = image.load_img(patch, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = x.astype('float32')/255
  pred = np.argmax(model.predict(x))

  print("isso é um = {} ".format(class_labels[pred]))

In [53]:
!wget url -P /content/macacos/ https://www.worldlifeexpectancy.com/images/a/w/b/alouatta-palliata/alouatta-palliata.jpg

--2020-05-02 23:13:37--  http://url/
Resolving url (url)... failed: Name or service not known.
wget: unable to resolve host address ‘url’
--2020-05-02 23:13:37--  https://www.worldlifeexpectancy.com/images/a/w/b/alouatta-palliata/alouatta-palliata.jpg
Resolving www.worldlifeexpectancy.com (www.worldlifeexpectancy.com)... 69.16.220.30
Connecting to www.worldlifeexpectancy.com (www.worldlifeexpectancy.com)|69.16.220.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124680 (122K) [image/jpeg]
Saving to: ‘/content/macacos/alouatta-palliata.jpg’

alouatta-palliata.j 100%[===================>] 121.76K   233KB/s    in 0.5s    

2020-05-02 23:13:38 (233 KB/s) - ‘/content/macacos/alouatta-palliata.jpg’ saved [124680/124680]

FINISHED --2020-05-02 23:13:38--
Total wall clock time: 1.2s
Downloaded: 1 files, 122K in 0.5s (233 KB/s)


In [56]:
check('/content/macacos/42969655324_869b5082da_b.jpg')

isso é um = mantled_howler 
